In [7]:
pip install scipy pulp


  Using cached PuLP-2.9.0-py3-none-any.whl.metadata (5.4 kB)
Using cached PuLP-2.9.0-py3-none-any.whl (17.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
from scipy.optimize import linprog
from pulp import LpProblem, LpVariable, LpMinimize, value
import time

# Problem definition for scipy.optimize.linprog
def solve_with_scipy():
    # Coefficients for the objective function (minimize cost: 2x1 + 3x2)
    c = [2, 3]

    # Inequality matrix and vector (A_ub * [x1, x2] >= b_ub)
    A = [
        [-3, -2],   # 3x1 + 2x2 >= 12 --> -3x1 - 2x2 <= -12
        [-1, -5],   # x1 + 5x2 >= 10  --> -x1 - 5x2 <= -10
        [-2, -4],   # 2x1 + 4x2 >= 16 --> -2x1 - 4x2 <= -16
        [-2, -2],   # 2x1 + 2x2 >= 10 --> -2x1 - 2x2 <= -10
        [-1, -0.5]  # x1 + 0.5x2 >= 3 --> -x1 - 0.5x2 <= -3
    ]
    b = [-12, -10, -16, -10, -3]

    # Bounds for x1 and x2 (non-negative variables)
    bounds = [(0, None), (0, None)]

    # Solve using linprog
    start_time = time.time()
    result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')
    elapsed_time = time.time() - start_time

    return {
        "x1": result.x[0],
        "x2": result.x[1],
        "Cost": result.fun,
        "Time": elapsed_time
    }

# Problem definition for PuLP
def solve_with_pulp():
    # Define the problem
    problem = LpProblem("Diet_Minimization", LpMinimize)

    # Define variables
    x1 = LpVariable("x1", lowBound=0)  # Amount of F1
    x2 = LpVariable("x2", lowBound=0)  # Amount of F2

    # Objective function: minimize cost
    problem += 2 * x1 + 3 * x2, "Total Cost"

    # Constraints
    problem += 3 * x1 + 2 * x2 >= 12, "Protein"
    problem += x1 + 5 * x2 >= 10, "Fat"
    problem += 2 * x1 + 4 * x2 >= 16, "Carbohydrates"
    problem += 2 * x1 + 2 * x2 >= 10, "Water"
    problem += x1 + 0.5 * x2 >= 3, "Vitamins"

    # Solve the problem
    start_time = time.time()
    problem.solve()
    elapsed_time = time.time() - start_time

    return {
        "x1": value(x1),
        "x2": value(x2),
        "Cost": value(problem.objective),
        "Time": elapsed_time
    }

# Compare solutions
scipy_result = solve_with_scipy()
pulp_result = solve_with_pulp()

# Print results
print("Scipy.optimize.linprog Solution:")
print(f"x1 (F1): {scipy_result['x1']}")
print(f"x2 (F2): {scipy_result['x2']}")
print(f"Cost: {scipy_result['Cost']}")
print(f"Time: {scipy_result['Time']} seconds\n")

print("PuLP Solution:")
print(f"x1 (F1): {pulp_result['x1']}")
print(f"x2 (F2): {pulp_result['x2']}")
print(f"Cost: {pulp_result['Cost']}")
print(f"Time: {pulp_result['Time']} seconds\n")

# Check if solutions are close
import numpy as np
if np.allclose([scipy_result['x1'], scipy_result['x2']], [pulp_result['x1'], pulp_result['x2']]):
    print("The solutions from Scipy and PuLP match.")
else:
    print("The solutions from Scipy and PuLP differ.")


Scipy.optimize.linprog Solution:
x1 (F1): 2.0
x2 (F2): 3.0
Cost: 13.0
Time: 0.012412071228027344 seconds

PuLP Solution:
x1 (F1): 2.0
x2 (F2): 3.0
Cost: 13.0
Time: 0.2971689701080322 seconds

The solutions from Scipy and PuLP match.


In [9]:
from scipy.optimize import linprog
import numpy as np

# Cost matrix (flattened for linprog)
c = [
    12, 6, 7, 5, 9,  # Costs for Source 1
    4, 5, 11, 3, 16, # Costs for Source 2
    15, 9, 6, 5, 9,  # Costs for Source 3
    12, 10, 6, 4, 5  # Costs for Source 4
]

# Supply constraints (rows in the cost matrix)
supply = [85, 110, 65, 80]

# Demand constraints (columns in the cost matrix)
demand = [90, 70, 70, 60, 50]

# Number of variables
num_sources = len(supply)
num_destinations = len(demand)
num_variables = num_sources * num_destinations

# Create inequality matrices for supply constraints (Ax <= b)
A_supply = np.zeros((num_sources, num_variables))
for i in range(num_sources):
    A_supply[i, i * num_destinations:(i + 1) * num_destinations] = 1

# Create inequality matrices for demand constraints (transpose for rows -> columns)
A_demand = np.zeros((num_destinations, num_variables))
for j in range(num_destinations):
    A_demand[j, j::num_destinations] = 1

# Combine all constraints
A = np.vstack([A_supply, -A_demand])
b = np.hstack([supply, -np.array(demand)])

# Bounds for all variables (x_ij >= 0)
bounds = [(0, None)] * num_variables

# Solve the linear programming problem
result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')

# Extract the solution
transportation_plan = result.x.reshape((num_sources, num_destinations))
total_cost = result.fun

# Display the transportation plan and total cost
print("Optimal Transportation Plan (amounts transported from sources to destinations):")
for i, row in enumerate(transportation_plan):
    print(f"Source {i + 1}: {row}")
print(f"\nMinimum Transportation Cost: {total_cost}")


Optimal Transportation Plan (amounts transported from sources to destinations):
Source 1: [ 0. 70.  0. 15.  0.]
Source 2: [90.  0.  0. 20.  0.]
Source 3: [ 0.  0. 65.  0.  0.]
Source 4: [ 0.  0.  5. 25. 50.]

Minimum Transportation Cost: 1685.0
